In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import swat.cas.datamsghandlers as dmh
from swat import *
import dlpy
from dlpy import Sequential
from dlpy.layers import * 
from dlpy.model import Optimizer, AdamSolver, Sequence
from dlpy.timeseries import plot_timeseries
from dlpy.metrics import accuracy_score, roc_auc_score, confusion_matrix
from getpass import getpass
%matplotlib inline

# Parameters

In [2]:
period = 30
seq_len = 50

# Connect to SAS Viya

In [3]:
sas = CAS('localhost', 5570, 'sasdemo', getpass('Please enter your passwod:'))

Please enter your passwod:········


# Python - Prepare Train Data

In [4]:
train = sas.read_csv('/sasdata/pdm/PM_train.txt', sep=' ', casout=CASTable(name='train', replace=True))
print(train.columns)
print(train.shape)

NOTE: Cloud Analytic Services made the uploaded file available as table TRAIN in caslib CASUSER(sasdemo).
NOTE: The table TRAIN has been created in caslib CASUSER(sasdemo) from binary data uploaded to Cloud Analytic Services.
Index(['id', 'cycle', 'setting1', 'setting2', 'setting3', 's1', 's2', 's3',
       's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14',
       's15', 's16', 's17', 's18', 's19', 's20', 's21', 'Unnamed: 26',
       'Unnamed: 27'],
      dtype='object')
(20631, 28)


In [63]:
t = train.groupby('id')

In [64]:
t.max()

ERROR: The user-defined programming statements could not be parsed.
ERROR: Syntax error detected on line 1 at column 22.  Found :, expecting <, <=, =, >, >=, EQ, GE, GT, IN, LE, LT, NE, NOTIN, ^<, ^=, ^>, ~<, ~=, ~>.  
ERROR: Failure opening table 'TRAIN'
ERROR: The action stopped due to errors.


SWATError: The user-defined programming statements could not be parsed.

In [5]:
train['ttf'] = train.groupby(['id'])['cycle'].transform(max)-train['cycle']

ERROR: Parameter 'table' is required but was not specified.
ERROR: The action stopped due to errors.
ERROR: Syntax error detected on line 1 at column 22.  Found :, expecting <, <=, =, >, >=, EQ, GE, GT, IN, LE, LT, NE, NOTIN, ^<, ^=, ^>, ~<, ~=, ~>.  
ERROR: The user-defined programming statements could not be parsed.
ERROR: The action stopped due to errors.


SWATError: The user-defined programming statements could not be parsed.

In [ ]:
train = pd.read_csv('/sasdata/pdm/PM_train.txt',sep=' ')
train.drop(train.columns[[26,27]], axis=1, inplace=True)
train.shape

In [ ]:
train['ttf'] = train.groupby(['id'])['cycle'].transform(max)-train['cycle']
train.shape

In [ ]:
train['nearfailure'] = train['ttf'].apply(lambda x: 1 if x <= period else 0)
train['age'] = train['cycle']
train.shape

# Python - Prepare Test Data

In [ ]:
test = pd.read_csv('/sasdata/pdm/PM_test.txt',sep=' ')
test.drop(test.columns[[26,27]], axis=1, inplace=True)
test.shape

In [ ]:
more = pd.read_csv('/sasdata/pdm/PM_truth.txt',sep=' ')
more.drop(more.columns[1], axis=1, inplace=True)
more.columns = ['more']
more['id'] = more.index + 1
more.shape

In [ ]:
rul = pd.DataFrame(test.groupby('id')['cycle'].max()).reset_index()
rul.columns = ['id', 'max']
rul.shape

In [ ]:
more['rtf'] = more['more'] + rul['max']
more.shape

In [ ]:
more.drop('more', axis=1, inplace=True)
test = test.merge(more,on=['id'],how='left')
test['ttf'] = test['rtf'] - test['cycle']
test.drop('rtf', axis=1, inplace=True)
test.shape

In [ ]:
test['nearfailure'] = test['ttf'].apply(lambda x: 1 if x <= period else 0)
test['age'] = test['cycle']
test.shape

# Connect to SAS Viya

In [ ]:
sas = CAS('localhost', 5570, 'sasdemo', getpass('Please enter your passwod:'))

# Load Data

In [ ]:
ts_train = dlpy.TimeseriesTable.from_pandas(sas, train)

In [ ]:
ts_train.shape

# Preprocess Data

In [ ]:
series = ts_train.columns.drop(['id','cycle','ttf']).values.tolist()

In [ ]:
ts_train.timeseries_formatting(timeid='cycle',
                               timeseries=series)

In [ ]:
ts_train.timeseries_accumlation(acc_interval='obs',
                                groupby='id')

In [ ]:
ts_train.prepare_subsequences(seq_len=seq_len, 
                              target='nearfailure', 
                              predictor_timeseries=series)

In [ ]:
ts_train.shape

# Model 1

In [ ]:
model = Sequential(sas, model_table='lstm_rnn')

model.add(InputLayer(std='STD'))
model.add(Recurrent(rnn_type='LSTM', output_type='samelength', n=100, dropout=0.1, act='sigmoid'))
model.add(Recurrent(rnn_type='LSTM', output_type='encoding',  n=50, dropout=0.1, act='sigmoid'))
model.add(OutputLayer(n=1, full_connect=True, act='sigmoid', error='entropy'))

model.plot_network()

In [ ]:
optimizer = Optimizer(algorithm=AdamSolver(), 
                      max_epochs=100, 
                      mini_batch_size=200,
                      seed=1234)

In [ ]:
seq_spec  = Sequence(**ts_train.sequence_opt)

In [ ]:
result = model.train(ts_train, 
                     optimizer=optimizer,
                     sequence=seq_spec, 
                     **ts_train.inputs_target,
                     model='lstm_rnn',
                     model_weights=sas.CASTable('rnn_weights', replace=True))

In [ ]:
model.plot_training_history(tick_frequency=1)

In [ ]:
result

# Model 2

In [ ]:
name = 'lstm_002'

In [ ]:
model = Sequential(sas, model_table=name)

model.add(InputLayer(std='STD'))
model.add(Recurrent(rnn_type='LSTM', output_type='samelength', n=100, dropout=0.1, act='tanh'))
model.add(Recurrent(rnn_type='LSTM', output_type='encoding',  n=50, dropout=0.1, act='tanh'))
model.add(OutputLayer(n=1, full_connect=True, act='sigmoid', error='entropy'))

model.plot_network()

In [ ]:
optimizer = Optimizer(algorithm=AdamSolver(), 
                      max_epochs=100, 
                      mini_batch_size=200,
                      seed=1234)

In [ ]:
seq_spec  = Sequence(**ts_train.sequence_opt)

In [ ]:
result = model.train(ts_train, 
                     optimizer=optimizer,
                     sequence=seq_spec, 
                     **ts_train.inputs_target,
                     model=name,
                     model_weights=name+'_weights')

In [ ]:
# model.plot_training_history(tick_frequency=1)
result.OptIterHistory[['FitError', 'Loss']][0:].plot(figsize=(12,5))
ax = plt.gca()
ax.get_xaxis().set_tick_params(labelsize=14)
ax.get_yaxis().set_tick_params(labelsize=14)
ax.legend(['Fit Error', 'Fit Loss'], loc='upper right', prop={'size': 14})
ax.set_xlabel('Iterations',fontsize=16)
ax.set_ylabel('Error / Loss',fontsize=16)
plt.show()

In [ ]:
result

# Evaluate

In [ ]:
ts_test = dlpy.TimeseriesTable.from_pandas(sas, test)

In [ ]:
ts_test.shape

In [ ]:
ts_test.timeseries_formatting(timeid='cycle',
                               timeseries=series)

In [ ]:
ts_test.timeseries_accumlation(acc_interval='obs',
                                groupby='id')

In [ ]:
ts_test.prepare_subsequences(seq_len=seq_len, 
                              target='nearfailure', 
                              predictor_timeseries=series)

In [ ]:
ts_test.shape

In [ ]:
model.score(ts_test,
            model=name,
            init_weights=name+'_weights', 
            copy_vars=['id','cycle','nearfailure'],
            casout=dict(name='lstm_pred', replace=True))

In [ ]:
pred = sas.CASTable('lstm_pred')

In [ ]:
sas.loadactionset('fedSql')

In [ ]:
sas.fedsql.execdirect('create table lstm_pred {options replace=true} as select '
                      '   *, '
                      '   case when _DL_Pred_ < 0.5 then 0 else 1 end as nf_pred '
                      'from lstm_pred')

In [ ]:
acc = accuracy_score(ts_test.nearfailure, pred.nf_pred, id_vars='id')
auc = roc_auc_score(ts_test.nearfailure, pred._DL_Pred_, id_vars='id', pos_label=1)
cmx = confusion_matrix(ts_test.nearfailure, pred.nf_pred, id_vars='id', labels=[0,1])

In [ ]:
print('Accuracy Score: %.4f' % acc)
print('AUC Score: %.4f' % auc)
print('\nConfusion Matrix:\n%s' % cmx)

# Save Model

In [ ]:
model.save_to_table('/sasdata/pdm')